<a href="https://colab.research.google.com/github/shemi024/ANN/blob/main/imbalanced_testing_BANK_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt 
import numpy as np
%matplotlib inline

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/breast-cancer_csv (1).csv')
df.sample(5)

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
204,40-49,premeno,25-29,0-2,no,1,left,right_low,no,no-recurrence-events
285,50-59,ge40,40-44,0-2,no,3,left,right_up,no,no-recurrence-events
65,50-59,ge40,15-19,0-2,no,1,right,central,no,no-recurrence-events
178,40-49,premeno,25-29,0-2,no,2,right,left_low,no,recurrence-events
198,40-49,premeno,15-19,12-14,no,3,right,right_low,yes,no-recurrence-events


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.dtypes


age            object
menopause      object
tumor-size     object
inv-nodes      object
node-caps      object
deg-malig       int64
breast         object
breast-quad    object
irradiat       object
Class          object
dtype: object

In [ ]:
def print_unique_col_values(df):
  for column in df:
    if df[column].dtypes=='object':
      print(f'{column}: {df[column].unique()}')
      

In [ ]:
print_unique_col_values(df)

age: ['40-49' '50-59' '60-69' '30-39' '70-79' '20-29']
menopause: ['premeno' 'ge40' 'lt40']
tumor-size: ['15-19' '35-39' '30-34' '25-29' '40-44' '10-14' '0-4' '20-24' '45-49'
 '50-54' '5-9']
inv-nodes: ['0-2' '3-5' '15-17' '6-8' '9-11' '24-26' '12-14']
node-caps: ['yes' 'no' nan]
breast: ['right' 'left']
breast-quad: ['left_up' 'central' 'left_low' 'right_up' 'right_low' nan]
irradiat: ['no' 'yes']
Class: ['recurrence-events' 'no-recurrence-events']


In [ ]:
#df1=pd.get_dummies(data=df, columns=['Geography','Gender'])
#df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
# scaling dataset


In [ ]:
for i in df:
  if df[i].dtype==np.dtype('O'):
    df[i]=le.fit_transform(df[i])

In [ ]:
df.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,2,2,2,0,1,3,1,2,0,1
1,3,0,2,0,0,1,1,0,0,0
2,3,0,6,0,0,2,0,1,0,1
3,2,2,6,0,1,3,1,1,1,0
4,2,2,5,4,1,2,0,4,0,1


In [ ]:
X = df.drop('Class',axis='columns')
y = df['Class']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=10,stratify=y)

In [ ]:
X_train.shape

(228, 9)

In [ ]:
X_test.shape

(58, 9)

In [ ]:
# Build a model (ANN) in tensorflow/keras
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
def ANN(X_train,y_train,X_test,y_test,loss):
  model = keras.Sequential([
                          keras.layers.Dense(58, input_dim=58, activation='relu'),
                          keras.layers.Dense(9, activation='relu'),
                          keras.layers.Dense(1, activation='sigmoid'),
  ])
  model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'])
  
  
  model.fit(X_train,y_train, epochs=10)
 
  # model.fit(X_train,y_train, epochs=10, class_weight=weights)

  print(model.evaluate(X_test,y_test))

  y_preds = model.predict(X_test)
  y_preds = np.round(y_preds)

  print("classification report: \n", classification_report(y_test,y_preds))

  return y_preds

In [ ]:
y_preds= ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/10


ValueError: ignored

In [ ]:
count_class_0,count_class_1 = df1.Exited.value_counts()

# divide by class

df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)


print("Random under- sampling:")
print(df_test_under.Exited.value_counts())

Random under- sampling:
0    2037
1    2037
Name: Exited, dtype: int64


In [ ]:
X=df_test_under.drop('Exited',axis='columns')
y=df_test_under['Exited']

# stratify used too balanced samples

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=15, stratify=y)

In [ ]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [ ]:
y_preds= ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 2ms/step - loss: 0.6761 - accuracy: 0.5842
Epoch 2/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6454 - accuracy: 0.6425
Epoch 3/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6304 - accuracy: 0.6536
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6204 - accuracy: 0.6594
Epoch 5/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6107 - accuracy: 0.6692
Epoch 6/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6026 - accuracy: 0.6800
Epoch 7/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5953 - accuracy: 0.6836
Epoch 8/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5904 - accuracy: 0.6904
Epoch 9/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5835 - accuracy: 0.6925
Epoch 10/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5785 - accura

In [ ]:
# Method2 = oversampling
# oversample 1-class and concat the dataframes of both clases

df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0,df_class_1_over],axis=0)

print("Random over- sampling:")
print(df_test_over.Exited.value_counts())

Random over- sampling:
0    7963
1    7963
Name: Exited, dtype: int64


In [ ]:
X=df_test_under.drop('Exited',axis='columns')
y=df_test_under['Exited']

# stratify used too balanced samples

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=15, stratify=y)

In [ ]:
y_preds= ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 2ms/step - loss: 0.6797 - accuracy: 0.5827
Epoch 2/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6479 - accuracy: 0.6422
Epoch 3/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6252 - accuracy: 0.6655
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6104 - accuracy: 0.6760
Epoch 5/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6015 - accuracy: 0.6836
Epoch 6/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5915 - accuracy: 0.6922
Epoch 7/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5832 - accuracy: 0.6965
Epoch 8/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5769 - accuracy: 0.7017
Epoch 9/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5704 - accuracy: 0.7070
Epoch 10/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5637 - accura

In [ ]:
# methode  smote
# to install imbalanced learn library use pip install imbalanced-learn command

X = df1.drop('Exited',axis='columns')
y = df1['Exited']

In [ ]:
from imblearn.over_sampling import SMOTE

smote=SMOTE(sampling_strategy='minority')
X_sm,y_sm=smote.fit_resample(X,y)

y_sm.value_counts()


1    7963
0    7963
Name: Exited, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=15, stratify=y)

In [ ]:
y_preds= ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.4922 - accuracy: 0.7961
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4710 - accuracy: 0.7956
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4576 - accuracy: 0.7977
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4442 - accuracy: 0.8052
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4301 - accuracy: 0.8145
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4199 - accuracy: 0.8171
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4114 - accuracy: 0.8196
Epoch 8/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3988 - accuracy: 0.8296
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3878 - accuracy: 0.8322
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3776 - accura

In [ ]:
# method4: use of ensemble with undersampling
df1.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [ ]:
X = df1.drop('Exited',axis='columns')
y = df1['Exited']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=15, stratify=y)

In [ ]:
df2=X_train.copy()
df2['Exited']=y_train

In [ ]:
df2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
5710,0.856,0.216216,0.5,0.554265,0.333333,0,0,0.339721,1,0,0,0,1,0
3745,0.852,0.256757,0.1,0.371163,0.333333,1,1,0.980432,0,1,0,1,0,0
5429,0.664,0.405405,0.7,0.000000,0.333333,1,0,0.325318,1,0,0,1,0,0
551,0.648,0.391892,0.6,0.426077,0.000000,1,1,0.010339,0,1,0,0,1,1
8967,0.970,0.094595,0.7,0.000000,0.333333,1,1,0.417230,1,0,0,0,1,0


In [ ]:
df2_class0=df2[df2.Exited == 0]
df2_class1=df2[df2.Exited == 1]

In [ ]:
def get_train_batch(df_majority, df_minority, start,end):
  df_train=pd.concat([df_majority[start:end], df_minority], axis=0)


  X_train=df_train.drop('Exited',axis='columns')
  y_train=df_train.Exited
  return X_train,y_train 

In [ ]:
X_train, y_train =get_train_batch(df2_class0, df2_class1, 0, 1495)

y_pred1=ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 3s 4ms/step - loss: 0.6830 - accuracy: 0.5702
Epoch 2/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6637 - accuracy: 0.6138
Epoch 3/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6479 - accuracy: 0.6298
Epoch 4/100
98/98 [==============================] - 0s 4ms/step - loss: 0.6334 - accuracy: 0.6448
Epoch 5/100
98/98 [==============================] - 0s 4ms/step - loss: 0.6221 - accuracy: 0.6605
Epoch 6/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6117 - accuracy: 0.6730
Epoch 7/100
98/98 [==============================] - 0s 3ms/step - loss: 0.6005 - accuracy: 0.6902
Epoch 8/100
98/98 [==============================] - 0s 3ms/step - loss: 0.5892 - accuracy: 0.6982
Epoch 9/100
98/98 [==============================] - 0s 4ms/step - loss: 0.5768 - accuracy: 0.7139
Epoch 10/100
98/98 [==============================] - 0s 4ms/step - loss: 0.5669 - accuracy: 0.7197
Epoch 11/

In [ ]:
X_train, y_train =get_train_batch(df2_class0, df2_class1, 1495,2990)

y_pred2=ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 2ms/step - loss: 0.6965 - accuracy: 0.5267
Epoch 2/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6663 - accuracy: 0.5949
Epoch 3/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6477 - accuracy: 0.6320
Epoch 4/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6315 - accuracy: 0.6518
Epoch 5/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6175 - accuracy: 0.6643
Epoch 6/100
98/98 [==============================] - 0s 2ms/step - loss: 0.6067 - accuracy: 0.6749
Epoch 7/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5979 - accuracy: 0.6845
Epoch 8/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5894 - accuracy: 0.6880
Epoch 9/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5827 - accuracy: 0.6954
Epoch 10/100
98/98 [==============================] - 0s 2ms/step - loss: 0.5748 - accuracy: 0.7053
Epoch 11/

In [ ]:
X_train, y_train =get_train_batch(df2_class0, df2_class1,2990,4130)

y_pred3=ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
87/87 [==============================] - 1s 2ms/step - loss: 0.6944 - accuracy: 0.5588
Epoch 2/100
87/87 [==============================] - 0s 2ms/step - loss: 0.6407 - accuracy: 0.6386
Epoch 3/100
87/87 [==============================] - 0s 2ms/step - loss: 0.6175 - accuracy: 0.6603
Epoch 4/100
87/87 [==============================] - 0s 2ms/step - loss: 0.6035 - accuracy: 0.6834
Epoch 5/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5933 - accuracy: 0.6903
Epoch 6/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5859 - accuracy: 0.6910
Epoch 7/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5784 - accuracy: 0.6935
Epoch 8/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5717 - accuracy: 0.7065
Epoch 9/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5639 - accuracy: 0.7108
Epoch 10/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5586 - accuracy: 0.7152
Epoch 11/

In [ ]:
y_pred_final=y_pred1.copy()
for i in range(len(y_pred1)):
  n_ones=y_pred1[i]+y_pred2[i]+y_pred3[i]
  if n_ones>1:
    y_pred_final[i]=1
  else:
    y_pred_final[i]=0

In [ ]:
cl_rep=classification_report(y_test,y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.94      0.74      0.83      1593
           1       0.44      0.80      0.57       407

    accuracy                           0.75      2000
   macro avg       0.69      0.77      0.70      2000
weighted avg       0.84      0.75      0.78      2000

